In [1]:
from unidecode import unidecode
import pandas as pd

In [2]:
df_to_join = pd.read_csv('../data/DOENCARESPIRATORIA/doencas_respiratorias.csv')

In [3]:
# Change - to 0 in all values
df_to_join = df_to_join.replace('-', 0)


In [4]:
df_to_join = df_to_join.drop(columns=['2025', 'Total'])

In [5]:
df_to_join

,Municipio,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,150010 ABAETETUBA,10225,10692,10629,9931,9364,7954,7296,7791,6366,7829,7941,7587,5514,10107,12251,12017,11926
1,150013 ABEL FIGUEIREDO,371,680,674,663,759,733,646,530,399,600,418,415,187,471,496,474,423
2,150020 ACARA,2680,2103,2000,2001,2086,2103,2083,1696,1498,1447,1440,1424,955,1369,1595,1399,1756
3,150030 AFUA,1069,712,1422,1304,747,915,863,1029,817,1227,1122,950,840,713,962,808,1039
4,150034 AGUA AZUL DO NORTE,1695,1955,1882,1335,812,1204,1243,743,1132,1741,1175,1266,877,1065,933,799,1039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,150820 VIGIA,2045,2186,2296,2400,2433,2456,2462,2412,2211,1484,2026,1943,1202,1231,1867,1603,1481
123,150830 VISEU,2103,2639,2790,3002,3146,2779,2697,2717,2939,2717,2920,3207,2055,2241,2299,2275,2278
124,150835 VITORIA DO XINGU,0,0,297,439,22,106,236,384,978,933,986,1337,1212,1487,1549,1495,1904
125,150840 XINGUARA,3908,3909,4623,3056,2137,3034,3625,3486,2763,2763,2888,2954,2361,2623,2937,3572,3126


In [6]:
df_full = pd.read_csv('../data/RESULTADOS/df_final.csv')
df_full = df_full.sort_values(by=['Município', 'Ano'])

In [7]:
new_column_name = 'Internações por Doenças Respiratórias'

In [8]:
# --- 2. Prepare the "Data to Join" DataFrame ---

# Drop the 'Total' row at the bottom
df_to_join = df_to_join[df_to_join['Municipio'] != 'Total'].copy()

# Extract only the name part from the 'Municipio' column
df_to_join['Municipio_Nome'] = df_to_join['Municipio'].str.split(' ', n=1).str[1]

# Identify the year columns to melt (unpivot)
year_columns = [col for col in df_to_join.columns if col.isdigit()]

# Melt the dataframe from wide to long format
df_melted = pd.melt(
    df_to_join,
    id_vars=['Municipio_Nome'],
    value_vars=year_columns,
    var_name='Ano',
    value_name=new_column_name
)

# Convert the new 'Ano' column from string to integer to allow for a proper merge
df_melted['Ano'] = df_melted['Ano'].astype(int)


# --- 3. Clean Keys in Both DataFrames for Merging ---

# Define a simple function to clean the municipality names
def clean_name(name):
  # Convert to string just in case there are non-string values
  name = str(name).lower()
  # Use unidecode to remove accents (e.g., 'município' -> 'municipio')
  return unidecode(name)

# Apply the cleaning function to create a matching key in both dataframes
df_full['join_key'] = df_full['Município'].apply(clean_name)
df_melted['join_key'] = df_melted['Municipio_Nome'].apply(clean_name)


# --- 4. Merge the DataFrames ---

# Perform a 'left' merge to add the new data to the full dataframe
# We keep all rows from df_full and add data from df_melted where keys match
df_final = pd.merge(
    df_full,
    df_melted[['join_key', 'Ano', new_column_name]], # Select only necessary columns from df_melted
    on=['join_key', 'Ano'],
    how='left'
)

# --- 5. Final Cleanup ---

# Drop the temporary join key column as it's no longer needed
df_final = df_final.drop(columns=['join_key'])

In [10]:
df_final

,Município,Ano,Código IBGE,Desmatamento (km²),PIB per capita (R$),PIB (R$ 1.000),VAB Agropecuária (R$ 1.000),VAB Indústria (R$ 1.000),VAB Serviços (R$ 1.000),População,Focos de Queimada,Total de Benefícios Básicos (Bolsa Família),Área plantada soja (ha),Área plantada milho (ha),Total Rebanho (Bovino),Internações por Doenças Respiratórias
0,Abaetetuba,2008,1500107,1.242461,3325.16,458888.70,52808.71,44176.94,152107.26,138005,194,12404,0,50,4194,10225
1,Abaetetuba,2009,1500107,2.989674,3642.24,509254.35,62031.90,30267.13,174853.63,139819,190,14743,0,50,3438,10692
2,Abaetetuba,2010,1500107,3.828983,4138.79,583793.45,71932.59,31239.98,201654.94,141054,162,16672,0,80,2263,10629
3,Abaetetuba,2011,1500107,0.832087,4971.70,709883.61,88434.83,44167.31,247545.10,142785,130,18705,0,100,1527,9931
4,Abaetetuba,2012,1500107,2.240918,5526.11,798053.28,115890.04,36469.75,276895.20,144415,153,20719,0,150,1859,9364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2443,Óbidos,2020,1505106,42.423582,17167.44,897960.34,419448.00,56388.87,149339.06,52306,612,6374,0,100,149900,2355
2444,Óbidos,2021,1505106,29.569607,18408.13,965929.86,426321.31,106455.48,148004.56,52473,377,6148,0,100,160052,2338
2445,Óbidos,2022,1505106,31.668326,18852.65,984523.73,449684.86,71301.47,162168.50,52222,428,6681,0,100,172430,2279
2446,Óbidos,2023,1505106,23.694052,19906.75,1041401.35,477191.79,75716.97,170475.72,52314,1249,6810,0,50,180010,2414


In [9]:
df_final.to_csv('../data/RESULTADOS/df_final2.csv', sep=',', index=False, encoding='utf-8')